In [18]:
from qiskit import BasicAer, execute, QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
from math import pi, sqrt, exp

Similar to QPE in IQPE (Iterative quantum phase estimation) also estimate the phase (and hence eigenvalue) introduced by a unitray when it is applied to its eigenvector, however in QPE we use multiple qubits to measure phase and apply the differnt powers of unitary to same eigenvector from different measuring qubit which acts as control, the powers of eigenvalues are then kickbakced to control qubit. 

in IQPE however we use single qubit as control and try to measure a component of eigenvalue by rotating and adjusting phase in each iteration .        

In [19]:
# This function will take a phase factor theta (value between 0 to 1) as input and will return a control unitary
# gate that, when applied to the eignevector (|1> in this case) will kickback an eigenvalue of
# e^2pi*phs_factor to control qubit,  when the  eigenvalue is unknown, it can be found with the phase estimation 
def Control_phase_gate(phs_factor):
    phase = phs_factor*(2*pi)
    qc1 = QuantumCircuit(2)
    qc1.cp(phase,0,1)
    CU_q = qc1.to_gate()
    return CU_q

In [20]:
def get_bits(m_CU, phs_adjust) :
    qr_control = QuantumRegister(1,'control')
    qr_target = QuantumRegister(1,'target')
    cr_measure = ClassicalRegister(1,'meas_control')
    qc = QuantumCircuit(qr_control, qr_target, cr_measure)
        
    qc.h(0)
    qc.x(1)
    qc.append(m_CU, [0,1])
    qc.p(phs_adjust, qr_control[0])
    qc.h(0)
    qc.measure(qr_control, cr_measure)
    
    bkend = BasicAer.get_backend('qasm_simulator')
    job =  execute(qc, bkend, shots=100)
    cnts = job.result().get_counts()
#    print(cnts)
    max_val = max(cnts, key=cnts.get)
    max_val_dec = int(max_val, 2)

    return max_val_dec

Lets try to measure theta of 1/12 = 30 degrees, set iteration itr = 12 

In [39]:
itr=12
theta_factor = 1/12 # phase angle of 30 degree
CU = Control_phase_gate(theta_factor)   # set the unitary

phase_adjust = 0
last_bit = 0
bit_reg = []
factor  = 0

for i in range(itr) :
    m_CU = CU.power(2**(itr-1-i))
    phase_adjust = 0
    for j in range(i):
        phase_adjust += 2*pi*(bit_reg[j]/2**(1 + i - j)) * -1
    bit_val = get_bits(m_CU, phase_adjust)
    bit_reg.append(bit_val)
    factor += bit_val/2**(itr-i)

print(factor)
for w in reversed(bit_reg):
    print(w),

0.083251953125
0
0
0
1
0
1
0
1
0
1
0
1


In [40]:
# get the value in decimal and calculate the angle
angle = factor * 2 * pi
print(' angle value in radian and degrees' , angle, angle * 57)


 angle value in radian and degrees 0.5230874486690036 29.815984574133203
